In [ ]:
%load_ext autoreload
%autoreload 2
import os
from time import sleep
import numpy as np
import torch
import scipy
import pandas as pd
import random
from scipy import stats
from load_datasets import SynDataMultiSrc
from models_train import *
from evaluation import Evaluation

In [ ]:
if torch.cuda.is_available():
  # torch.set_default_tensor_type('torch.cuda.FloatTensor')
  print('Use ***GPU***')
  print(torch.cuda.get_device_properties(0).total_memory/1024/1024/1024,'GB')
else:
  print('Use CPU')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Use ***GPU***
14.726318359375 GB


# Train the model

In [ ]:
# Configuration
OUTPUT_DIR = 'save_outputs/save_multi_source_with_validation_F'
dim_z = 45
training_iter = 5000
transfer_flag = FLAGS_LEARN_TRANSFER
n_samples = 100
reg_alpha_w, lr_w = 1e-3, 1e-3
reg_alpha_y, reg_noise_y, lr_y =1e-2, 1e-1, 1e-3
reg_alpha_zy, lr_zy =1e-1, 1e-2
N_target_train = 50

# Lists to store results
learn_transfer_trans_factor_lst = []
learn_transfer_test_stats_lst = []
learn_transfer_test_stats_w_lst = []
learn_transfer_test_stats_y_lst = []

# Load data
source_size = 1000
dataset = SynDataMultiSrc(source_size=source_size)

# Create output directory
if not os.path.exists(OUTPUT_DIR):
  os.makedirs(OUTPUT_DIR)

# Train the models
# for m in [1,2,3,4]: # This dataset (SynDataMultiSrc) has maximum 4 sources
for m in [1,2]:
  loss_lst = []
  trans_factors = []
  test_stats = []
  test_stats_w = []
  test_stats_y = []
  for i, (datas, datat) in enumerate(dataset.get_source_target(m_sources=m)):
    print('======================================================================================')
    print('#Source: {}, Replicate: {}'.format(m, i+1))
    print('======================================================================================')
    sleep(0.5)
    # Source data to tensor
    Ws, Ys, Y_cfs, mus, Xs = datas[0][1].reshape(-1), datas[0][2].reshape(-1), datas[1][0].reshape(-1),\
                              np.concatenate((datas[1][1],datas[1][2]),axis=1), datas[0][0]
    Ts = len(Ys)
    xs = torch.from_numpy(Xs.reshape(Ts,-1)).float().to(device)
    ys = torch.from_numpy(Ys.reshape(-1,1)).float().to(device)
    ws = torch.from_numpy(Ws.reshape(-1,1)).float().to(device)

    # Target train data to tensor
    Wt, Yt, Y_cft, mut, Xt = datat[0][1].reshape(-1)[:N_target_train],\
                              datat[0][2].reshape(-1)[:N_target_train],\
                              datat[1][0].reshape(-1)[:N_target_train],\
                              np.concatenate((datat[1][1],datat[1][2]),axis=1)[:N_target_train],\
                              datat[0][0][:N_target_train,:]
    Tt = len(Yt)
    xt = torch.from_numpy(Xt.reshape(Tt,-1)).float().to(device)
    yt = torch.from_numpy(Yt.reshape(-1,1)).float().to(device)
    wt = torch.from_numpy(Wt.reshape(-1,1)).float().to(device)

    # Target test data to tensor
    Wtte, Ytte, Y_cftte, mutte, Xtte = datat[0][1].reshape(-1)[N_target_train:-100],\
                                        datat[0][2].reshape(-1)[N_target_train:-100],\
                                        datat[1][0].reshape(-1)[N_target_train:-100],\
                                        np.concatenate((datat[1][1],datat[1][2]),axis=1)[N_target_train:-100],\
                                        datat[0][0][:,:][N_target_train:-100]
    Ttte = len(Ytte)
    xtte = torch.from_numpy(Xtte.reshape(Ttte,-1)).float().to(device)
    ytte = torch.from_numpy(Ytte.reshape(-1,1)).float().to(device)
    wtte = torch.from_numpy(Wtte.reshape(-1,1)).float().to(device)

    # Train
    source_ranges = [(idx, idx+source_size) for idx in range(0,m*source_size,source_size)]
    domain_ranges =  source_ranges + [(m*source_size, m*source_size+N_target_train)]
    print('*** P(W|X)')
    model_w = trainW(train_x=torch.cat((xs,xt),dim=0),
                     train_w=torch.cat((ws,wt),dim=0).reshape(-1),
                     n_domains=m+1, domain_ranges=domain_ranges,
                     training_iter=training_iter, transfer_flag=transfer_flag,
                     reg_alpha=reg_alpha_w, lr=lr_w)
    w_samples = model_w.sample(x=xtte,n_samples=n_samples)

    print('*** P(Y|X,W)')
    model_y = trainY(train_x=torch.cat((xs,xt),dim=0),
                     train_y=torch.cat((ys,yt),dim=0),
                     train_w=torch.cat((ws,wt),dim=0),
                     n_domains=m+1, domain_ranges=domain_ranges,
                     training_iter=training_iter, transfer_flag=transfer_flag,
                     reg_alpha=reg_alpha_y, reg_noise=reg_noise_y, lr=lr_y)
    y_samples_xw = model_y.sample(x=xtte,w_samples=w_samples,n_samples=n_samples)

    print('*** P(Z|Y,X,W) and P(Y|W,Z)')
    model_zy = trainZY(train_x=torch.cat((xs,xt),dim=0),
                       train_y=torch.cat((ys,yt),dim=0),
                       train_w=torch.cat((ws,wt),dim=0),
                       n_domains=m+1, domain_ranges=domain_ranges, dim_z=dim_z,
                       training_iter=training_iter, transfer_flag=transfer_flag,
                       reg_alpha=reg_alpha_zy, lr=lr_zy)
    
    
    # Evaluate P(W|X)
    accur = torch.sum((model_w.pred(xtte, domain_ranges=[(0,0)]*m+[(0,xtte.shape[0])])>0)*1.0==wtte.reshape(-1))/len(xtte)
    test_stats_w.append(np.insert(model_w.trans_factor().cpu().detach().numpy(),0,accur.cpu().detach().numpy()))

    # Evaluate P(Y|X,W)
    mae = torch.mean(torch.abs(model_y.pred(xtte, wtte, domain_ranges=[(0,0)]*m+[(0,xtte.shape[0])]).reshape(-1) - ytte.reshape(-1)))
    rmse = torch.sqrt(torch.mean((model_y.pred(xtte, wtte, domain_ranges=[(0,0)]*m+[(0,xtte.shape[0])]).reshape(-1) - ytte.reshape(-1))**2))
    test_stats_y.append(np.concatenate((np.asarray([mae.cpu().detach().numpy(), rmse.cpu().detach().numpy()]),
                                        model_y.trans_factor().cpu().detach().numpy())))

    # Evaluate ATE, ITE
    y_samples_do0,_ = model_zy.sample_v2(xtte, do_w=torch.zeros((xtte.shape[0],1),device=device),
                                         y_samples=y_samples_xw, w_samples=w_samples, n_samples=n_samples)
    y_samples_do1,_ = model_zy.sample_v2(xtte, do_w=torch.ones((xtte.shape[0],1),device=device),
                                         y_samples=y_samples_xw, w_samples=w_samples, n_samples=n_samples)
    
    y_do0_mean = torch.mean(y_samples_do0,dim=1)
    y_do1_mean = torch.mean(y_samples_do1,dim=1)

    # Y0tte = (1-Wtte)*Ytte + Wtte*Y_cftte
    # Y1tte = Wtte*Ytte + (1-Wtte)*Y_cftte
    eval = Evaluation(m0=mutte[:,0], m1=mutte[:,1])
    abs_err = eval.absolute_err_ate(y_do0_mean.cpu().detach().numpy(), y_do1_mean.cpu().detach().numpy())
    pehe = eval.pehe(y_do0_mean.cpu().detach().numpy(), y_do1_mean.cpu().detach().numpy())

    test_stats.append((abs_err, pehe))
    trans_factors.append(np.concatenate((model_w.trans_factor().cpu().detach().numpy(), model_y.trans_factor().cpu().detach().numpy(),
                                        model_zy.trans_factor().cpu().detach().numpy())))
    
    # Save output of replicate i+1
    np.savez(OUTPUT_DIR + '/graphs_multi_source_learn_transfer_stats_{}_replicate_{}.npz'.format(m,i+1),
            trans_factors=trans_factors,
            test_stats=test_stats,
            test_stats_w=test_stats_w,
            test_stats_y=test_stats_y)

  learn_transfer_trans_factor_lst.append(trans_factors)
  learn_transfer_test_stats_lst.append(test_stats)
  learn_transfer_test_stats_w_lst.append(test_stats_w)
  learn_transfer_test_stats_y_lst.append(test_stats_y)

  # Save outputs
  learn_transfer_stats_mean = np.array([np.mean(test_stats,axis=0) for test_stats in learn_transfer_test_stats_lst])
  learn_transfer_stats_stderr = np.array([stats.sem(test_stats,axis=0) for test_stats in learn_transfer_test_stats_lst])
  np.savez(OUTPUT_DIR + '/graphs_multi_source_learn_transfer_stats.npz',
          stats_mean=learn_transfer_stats_mean,
          stats_stderr=learn_transfer_stats_stderr)
  
  learn_transfer_trans_factor_mean = np.array([np.mean(test_stats,axis=0) for test_stats in learn_transfer_trans_factor_lst])
  learn_transfer_trans_factor_stderr = np.array([stats.sem(test_stats,axis=0) for test_stats in learn_transfer_trans_factor_lst])
  np.savez(OUTPUT_DIR + '/graphs_multi_source_learn_transfer_trans_factor.npz',
          stats_mean=learn_transfer_trans_factor_mean,
          stats_stderr=learn_transfer_trans_factor_stderr)
  
  learn_transfer_stats_w_mean = np.asarray([np.mean(test_stats,axis=0) for test_stats in  learn_transfer_test_stats_w_lst])
  learn_transfer_stats_w_stderr = np.asarray([stats.sem(test_stats,axis=0) for test_stats in  learn_transfer_test_stats_w_lst])
  np.savez(OUTPUT_DIR + '/graphs_multi_source_w_learn_transfer_stats.npz',
          stats_mean=learn_transfer_stats_w_mean,
          stats_stderr=learn_transfer_stats_w_stderr)
  
  learn_transfer_stats_y_mean = np.array([np.mean(test_stats,axis=0) for test_stats in learn_transfer_test_stats_y_lst])
  learn_transfer_stats_y_stderr = np.array([stats.sem(test_stats,axis=0) for test_stats in learn_transfer_test_stats_y_lst])
  np.savez(OUTPUT_DIR + '/graphs_multi_source_y_learn_transfer_stats.npz',
          stats_mean=learn_transfer_stats_y_mean,
          stats_stderr=learn_transfer_stats_y_stderr)

#Source: 1, Replicate: 1
*** P(W|X)
Iter 100/5000 - Loss: 1903.954   Transfer Ratio: [0.478]
Iter 200/5000 - Loss: 1351.080   Transfer Ratio: [0.462]
Iter 300/5000 - Loss: 997.589   Transfer Ratio: [0.452]
Iter 400/5000 - Loss: 758.526   Transfer Ratio: [0.447]
Iter 500/5000 - Loss: 593.460   Transfer Ratio: [0.447]
Iter 600/5000 - Loss: 480.814   Transfer Ratio: [0.449]
Iter 700/5000 - Loss: 403.407   Transfer Ratio: [0.453]
Iter 800/5000 - Loss: 347.639   Transfer Ratio: [0.459]
Iter 900/5000 - Loss: 304.977   Transfer Ratio: [0.465]
Iter 1000/5000 - Loss: 270.649   Transfer Ratio: [0.471]
Iter 1100/5000 - Loss: 242.019   Transfer Ratio: [0.478]
Iter 1200/5000 - Loss: 217.596   Transfer Ratio: [0.485]
Iter 1300/5000 - Loss: 196.510   Transfer Ratio: [0.492]
Iter 1400/5000 - Loss: 178.224   Transfer Ratio: [0.5]
Iter 1500/5000 - Loss: 162.376   Transfer Ratio: [0.507]
Iter 1600/5000 - Loss: 148.690   Transfer Ratio: [0.515]
Iter 1700/5000 - Loss: 136.919   Transfer Ratio: [0.522]
Iter

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:144: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:145: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  retur

### Print $\epsilon_{\text{ATE}}, \sqrt{\epsilon_{\text{PEHE}}}$

In [ ]:
learn_transfer_stats_mean = np.array([np.mean(test_stats,axis=0) for test_stats in learn_transfer_test_stats_lst])
learn_transfer_stats_stderr = np.array([stats.sem(test_stats,axis=0) for test_stats in learn_transfer_test_stats_lst])
learn_transfer_stats_mean

array([[0.36123179, 1.29829665],
       [0.18845618, 1.26419748]])

### Print transfer factors

In [ ]:
learn_transfer_trans_factor_mean = np.array([np.mean(test_stats,axis=0) for test_stats in learn_transfer_trans_factor_lst])
learn_transfer_trans_factor_stderr = np.array([stats.sem(test_stats,axis=0) for test_stats in learn_transfer_trans_factor_lst])
learn_transfer_trans_factor_mean

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


array([array([0.47815722, 0.4940784 , 0.6321679 ], dtype=float32),
       array([0.479457  , 0.40853563, 0.40444836, 0.4708232 , 0.46985704,
       0.2990765 , 0.47244328, 0.5006661 , 0.49880344], dtype=float32)],
      dtype=object)

### Print prediction accuracy of $P(W|X)$

In [ ]:
learn_transfer_stats_w_mean = np.asarray([np.mean(test_stats,axis=0) for test_stats in  learn_transfer_test_stats_w_lst])
learn_transfer_stats_w_stderr = np.asarray([stats.sem(test_stats,axis=0) for test_stats in  learn_transfer_test_stats_w_lst])
learn_transfer_stats_w_mean

/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


array([array([0.8922353 , 0.47815722], dtype=float32),
       array([0.8936471 , 0.479457  , 0.40853563, 0.40444836], dtype=float32)],
      dtype=object)

### Print prediction error of $P(Y|W,X)$

In [ ]:
learn_transfer_stats_y_mean = np.array([np.mean(test_stats,axis=0) for test_stats in learn_transfer_test_stats_y_lst])
learn_transfer_stats_y_stderr = np.array([stats.sem(test_stats,axis=0) for test_stats in learn_transfer_test_stats_y_lst])
learn_transfer_stats_y_mean

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


array([array([1.5923061, 2.4033668, 0.4940784], dtype=float32),
       array([1.5730364 , 2.3542314 , 0.4708232 , 0.46985704, 0.2990765 ],
      dtype=float32)], dtype=object)